please please please be the final version

# import stuff

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
import json
import os
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, ConfusionMatrixDisplay
from hpsklearn import svc
from sklearn.model_selection import train_test_split
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
import warnings

# prep data

In [52]:
# initial dataset stuff
nlp = spacy.load("spacy-twitter") # out of function so you don't load it every time (it takes a while)

# function for glove embeddings
def embed_dataset(dataset_text):
    encoded = np.array([nlp(text).vector for text in dataset_text])
    return encoded.tolist()

# function to load dataset from folder. Also embeds the text.
def get_dataset(name):
    """
    loads a dataset and embeds the text. text must be in column named "text".
    datasets are in the folder datasets/
    name must be a string that's matches the csv file in datasets
    """
    dataset = pd.read_csv(f'datasets/{name}.csv')
    dataset.rename(columns = {"Unnamed: 0":"entry"}, inplace=True) #the entry label never carries over
    dataset['e_text'] = embed_dataset(dataset['text'])
    return dataset

# train models

In [53]:
from hpsklearn import HyperoptEstimator
from hyperopt import hp

#function to optimize model with hyperopt
def optimize_model(model, X_train, y_train, evals):
    """
    optimizes a model with hyperopt. returns the optimized and fitted model.
    
    model: should match a sklearn model - list on github. can preset variables or set a range with hp
    X_train and y_train: training set
    evals: number of trials to do
    """
    mod = HyperoptEstimator(classifier=model,
                            preprocessing=[],
                            max_evals=evals,
                            trial_timeout=20,
                            algo=tpe.suggest,
                            verbose=False,
                            n_jobs=6)
    mod.fit(X_train, y_train, random_state=42)
    #print(mod.best_model())
    return mod

def evaluate_model(model, X_test, y_test):
    pred_y = model.predict(X_test)
    acc_mod = accuracy_score(y_test, pred_y)
    print("Accuracy:", float("{0:.2f}".format(acc_mod*100)), "%")
    f1_mod = f1_score(y_test, pred_y, average="macro")
    print("F1:", float("{0:.2f}".format(f1_mod*100)), "%")
    cm = confusion_matrix(y_test, pred_y)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["true", "false"])
    disp.plot()
    plt.show() 

In [54]:
def optimize_model_v2(search_space, objective, evals):
    trials = Trials()
    best_params = fmin(
        fn = objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=evals,
        timeout=120,
        trials=trials
    )
    set_params = space_eval(search_space, best_params)
    score = trials.best_trial['result']['loss']
    return set_params, score

In [55]:
# get breakdown of categories with confidence level, as well as dataset coverage
def get_category_info(dataset_name, dataset, confidence, size_limit):
    file_name = f"{dataset_name}_cats/{dataset_name}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    info = []
    dataset_coverage = np.zeros([len(dataset)+2], dtype=bool)
    for category in data.keys():
        cat_entries = [int(i) for i in data[category].keys() if data[category][i] > confidence]
        if len(cat_entries) < size_limit:
            print(f"Skipped category: {category} due to low numbers")
            continue
        info.append((category, len(cat_entries)))
        for entry in cat_entries:
            if dataset_coverage[entry] == False:
                dataset_coverage[entry] = True
    uniques, counts = np.unique(dataset_coverage, return_counts=True)
    """for i in range(len(dataset_coverage)):
        if dataset_coverage[i] == False:
            print(i)"""
    percentages = dict(zip(uniques, counts * 100 / len(dataset_coverage)))
    return info, percentages

In [56]:
def train_models_v2(dataset_name, train_set, confidence, size_limit, model_list):
    """
    trains a set of models in each category. returns the best model for each category, in the form {'category': [modelscore, modelname, fittedmodel]}

    dataset_name: a string with the name of the training set. used for calling the category file
    train_set: the training set to use
    confidence: the confidence required to consider an entry part of a category
    size_limit: the number of entries needed in a category to consider that category for training
    model_list: the list of models to train. in the form [("model_name1", optimization_function1, search_space1, model1), etc]
    """
    file_name = f"{dataset_name}_cats/{dataset_name}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    f.close()
    category_models = {} #this will be returned
    for category in data.keys(): 
        cat_entries = [int(i) for i in data[category].keys() if data[category][i] > confidence]
        
        # skip category if size of category is below limit
        if len(cat_entries) < size_limit:
            print(f"Skipped category: {category} due to low numbers")
            continue
        
        category_data = train_set.filter(axis=0, items=cat_entries)

        #split validation set
        X = category_data.drop('target', axis=1)
        y = category_data["target"]
        try:
            X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
        except:
            print(f"Skipped category: {category} due to class issues")
            continue

        # skip category if split only has one class
        if (len(np.unique(y_train)) <= 1):
            print(f"Skipped category: {category} due to class issues")
            continue

        X_train_text = np.array([text for text in X_train['e_text']])
        X_val_text = np.array([text for text in X_val['e_text']])

        trained_models = []
        # train models from list
        for model_name, search_space, mod in model_list:
            def objective(search_space):
                warnings.filterwarnings('ignore')
                model = mod.set_params(**search_space)
                model.fit(X_train_text, y_train)
                y_pred = model.predict(X_val_text)
                accuracy = accuracy_score(y_val, y_pred)
                return {'loss': -accuracy, 'status': STATUS_OK}
            try:
                best_params, score = optimize_model_v2(search_space, objective, 200)
                mod.set_params(**best_params)
                mod.fit(X_train_text, y_train)
                score *= -1
                trained_models.append((model_name, mod, score))
                #print(f"Trained {model_name} on {category}")
            except:
                print(f"Error training {model_name} in category {category}, skipping")
                continue
        
        #get the best model
        best_model = [0, "x", "x"]
        for name, model, score in trained_models:
            if score > best_model[0]:
                best_model = [score, name, model]
        print(f"Trained models on {category}, added {best_model[1]} to list")
        #add best model to list
        category_models[category] = best_model
    return category_models

In [57]:
def train_models(dataset_name, train_set, confidence, size_limit, model_list):
    """
    trains a set of models in each category. returns the best model for each category, in the form {'category': [modelscore, modelname, fittedmodel]}

    dataset_name: a string with the name of the training set. used for calling the category file
    train_set: the training set to use
    confidence: the confidence required to consider an entry part of a category
    size_limit: the number of entries needed in a category to consider that category for training
    model_list: the list of models to train. in the form [("model_name1", model1), ("model_name2", model2), etc]
    """
    file_name = f"{dataset_name}_cats/{dataset_name}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    f.close()
    category_models = {} #this will be returned
    for category in data.keys(): 
        cat_entries = [int(i) for i in data[category].keys() if data[category][i] > confidence]
        
        # skip category if size of category is below limit
        if len(cat_entries) < size_limit:
            print(f"Skipped category: {category} due to low numbers")
            continue
        
        category_data = train_set.filter(axis=0, items=cat_entries)

        #split validation set
        X = category_data.drop('target', axis=1)
        y = category_data["target"]
        # skip category if split only has one class
        if (len(np.unique(y)) <= 1):
            print(f"Skipped category: {category} due to class issues")
            continue
        try:
            X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
        except:
            print(f"Skipped category: {category} due to class issues")
            continue

        X_train_text = np.array([text for text in X_train['e_text']])

        trained_models = []
        # train models from list
        for model_name, model in model_list:
            try:
                optimized = optimize_model(model, X_train_text, y_train, 1)
                best_model = optimized.best_model()['learner'].fit(X_train_text, y_train)
                trained_models.append((model_name, best_model))
            except:
                print(f"Error training {model_name} in category {category}, skipping")
                continue
        
        #get the best model
        X_val_text = np.array([text for text in X_val['e_text']])
        best_model = [0, "x", "x"]
        for model_name, model in trained_models:
            score = model.score(X_val_text, y_val)
            if score > best_model[0]:
                best_model = [score, model_name, model]

        #add best model to list
        category_models[category] = best_model
    return category_models

# voting

In [58]:
def predict_points(trained_models, test_cat_file, X_test):
    """
    predict points using the trained models. returns an array of the predictions

    trained_models: the models trained in each category, in the form {category: [modelscore, modelname, fittedmodel]}
    test_cat_file: the filepath to the organised category file
    X_test: the test set X values
    """
    #load category data
    f = open(test_cat_file)
    category_data = json.load(f)
    f.close()

    # return arrays
    final_predictions = []
    
    #embedded_text = np.array([text for text in X_test['e_text']])
    for index, row in X_test.iterrows():
        test_point = np.array([row['e_text']])
        point_categories = category_data[str(index)]

        # get weights of each point's topic
        topic_weights = {}
        for category in point_categories:
            main_category = category.split("/")[1]
            if main_category not in trained_models.keys():
                continue
            if main_category in topic_weights:
                topic_weights[main_category] += point_categories[category]
            else:
                topic_weights[main_category] = point_categories[category]
        
        #make topic predictions
        model_predictions = []
        for category in topic_weights.keys():
            modelscore, modelname, model = trained_models[category]
            prediction = model.predict(test_point)
            model_predictions.append((prediction[0], modelscore, modelname, category))

        # aggregate predictions
        truefalse_scores = {True: 0, False:0}
        for prediction, modelscore, modelname, category in model_predictions:
            truefalse_scores[prediction] += modelscore
        
        #determine final prediction
        if truefalse_scores[True] > truefalse_scores[False]:
            final_predictions.append(True)
        else:
            final_predictions.append(False)
    return final_predictions

# evaluate results

In [59]:
def check_score(test, pred):
    acc = accuracy_score(test, pred)
    f1 = f1_score(test, pred, average="macro")
    #cm = confusion_matrix(test, pred)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["true", "false"])
    #disp.plot()
    plt.show() 
    return float("{0:.2f}".format(acc*100)), float("{0:.2f}".format(f1*100))

# pipeline

get dataset -> train models on dataset -> make predictions and vote -> evaluate results

In [60]:
def train_and_evaluate(train_set, test_set, confidence, size_limit, model_list):
    """
    Train and evaluate dataset on test set. Returns results and models
    train_set: training set. In the form ["dataset_name", file_reference_name, dataset]
    test_set: testing set(s). In the form [("dataset_name1", file_reference_name1, dataset1), ("dataset_name2", file_reference_name2, dataset2), etc]
    confidence: confidence threshold to consider a training point
    size_limit: size threshold to consider training a category
    model_list: list of models to train, in the form [("model_name1", model1), ("model_name2", model2), etc]
    """
    #prep dataset
    dataset_name = train_set[0]
    ref_name = train_set[1]
    dataset = train_set[2]
    X = dataset.drop("target", axis=1)
    y = dataset["target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
    training_set = pd.concat([X_train, y_train], axis=1)

    #get dataset coverage
    info, coverage = get_category_info(ref_name, dataset, confidence, size_limit)
    dataset_coverage = coverage[True]

    #train models
    trained_models = train_models(ref_name, training_set, confidence, size_limit, model_list)

    #make predictions on test set
    test_cat_file = f"{ref_name}_categories.json"
    predictions = predict_points(trained_models, test_cat_file, X_test)
    results = check_score(y_test, predictions)

    final_results = []
    final_results.append((dataset_name, results))

    #make predictions on out of domain test sets
    for set_name, ref_name, set_data in test_set:
        test_cat_file = f"{ref_name}_categories.json"
        X = set_data.drop("target", axis=1)
        y = set_data["target"]
        predictions = predict_points(trained_models, test_cat_file, set_data)
        results = check_score(y, predictions)
        final_results.append((set_name, results))
    
    return dataset_coverage, trained_models, final_results

In [61]:
def run_tests(tests, confidence, size_limit, model_list):
    dataset_coverage = []
    test_results = []
    trained_models = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        coverage, models, results = train_and_evaluate(train, t, confidence, size_limit, model_list)
        dataset_coverage.append((train[0], coverage))
        test_results.append((train[0], results))
        trained_models.append((train[0], models))
    return dataset_coverage, trained_models, test_results

# Prep Models

In [62]:
pheme = get_dataset('pheme')
X = pheme.drop("target", axis=1)
y = pheme["target"]
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, stratify=y) 
X_train_text = np.array([text for text in X_train['e_text']])
X_val_text = np.array([text for text in X_val['e_text']])

In [63]:
# SVM
from sklearn.svm import SVC

SVM_search_space={  
                'C': hp.lognormal('C', 0, 1),
                'kernel':hp.choice('kernel', ["linear", "poly", "rbf", "sigmoid"]),
                'coef0':hp.uniform('coef0', 0.0, 1.0),
                'shrinking':hp.choice('shrinking', [True, False]),
                'tol':hp.loguniform('tol', np.log(1e-5), np.log(1e-2)),
                'degree':hp.choice('degree', [1, 2, 3, 4, 5]),
                'gamma':hp.choice('gamma', ["scale", "auto"]),
                }


In [64]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

KNN_search_space={
                "n_neighbors":hp.choice('n_neighbors', np.arange(1, 16, dtype=int)),
                "algorithm":hp.choice("algorithm", ["auto", "ball_tree", "kd_tree", "brute"]),
                "metric": hp.choice("metric", ["cityblock", "l1", "l2", "minkowski", "euclidean", "manhattan"]),
                "p":hp.uniform("p", 1, 5)
                }

In [65]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
import warnings

LR_search_space={
                'C': hp.lognormal('C', 0, 1),
                'penalty':hp.choice('p_saga',['elasticnet','l1','l2',None]),
                'tol': hp.loguniform('tol',-13,-1),
                'l1_ratio': hp.uniform('l1_ratio',0,1)
                }

In [66]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

RF_search_space={  'n_estimators':hp.randint('n_estimators',200,1000),
                'max_depth': hp.randint('max_depth',10,200),                      
                'min_samples_split':hp.uniform('min_samples_split',0,1),   
                'min_samples_leaf':hp.randint('min_samples_leaf',1,10),            
                'criterion':hp.choice('criterion',['gini','entropy']),               
                'max_features':hp.choice('max_features',['sqrt', 'log2']) }


In [67]:
# MLP
from sklearn.neural_network import MLPClassifier

MLP_search_space={
                'activation':hp.choice('activation', ["identity","logistic","tanh","relu"]),
                'solver':hp.choice('solver', ['lbfgs', 'sgd', 'adam']),
                'alpha':hp.uniform("alpha", 1e-4, 0.01),
                'learning_rate':hp.choice('learning_rate', ['constant', 'invscaling', 'adaptive']),
                'learning_rate_init':hp.uniform("learning_rate_init", 1e-4, 0.1),
                'power_t':hp.uniform('power_t', 0.1, 0.9),
                'tol':hp.uniform('tol', 1e-4, 0.01),
                'momentum':hp.uniform('momentum', 0.8, 1.0),
                'early_stopping':hp.choice('early_stopping', [True, False]),
                'beta_1':hp.uniform("beta_1", 0.8, 1.0),
                'beta_2':hp.uniform("beta_2", 0.95, 1.0),
                'epsilon':hp.uniform("epsilon", 1e-9, 1e-5)
                }

In [68]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB

NB_search_space={
                'var_smoothing': 10**-9
                }


In [69]:
# SGD
from sklearn.linear_model import SGDClassifier

SGD_search_space={
                'loss':hp.choice('loss',["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]),
                'penalty':hp.choice("penalty", ["l2", "l1", "elasticnet", None]),
                'alpha':hp.loguniform("alpha", np.log(1e-6), np.log(1e-1)),
                "l1_ratio":hp.loguniform("l1_ratio", np.log(1e-7), np.log(1)),
                "tol":hp.loguniform("tol", np.log(1e-5), np.log(1e-2)),
                'learning_rate':hp.choice("learning_rate",  ["optimal", "invscaling", "constant", "adaptive"]),
                'eta0':hp.loguniform("eta0", np.log(1e-5), np.log(1e-1))
                }

In [70]:
from hpsklearn import sgd_classifier, k_neighbors_classifier, logistic_regression, svc, random_forest_classifier, mlp_classifier, gaussian_nb

sgd_loss = hp.choice("loss", {'hinge', 'squared_error', 'epsilon_insensitive', 'perceptron', 'modified_huber', 'squared_hinge', 'squared_epsilon_insensitive', 'huber', 'log_loss'})
model_list = [
    ("SVC", svc("SVC", random_state=42)),
    ("KNN", k_neighbors_classifier("knn")),
    ("Logistic Regression", logistic_regression("LR", random_state=42, solver="saga", penalty=hp.choice("penalty", {None, "l1", "l2"}))),
    ("Random Forest", random_forest_classifier("Random Forest", random_state=42)),
    ("MLP", mlp_classifier("MLP", random_state=42)),
    ("Gaussian NB", gaussian_nb("GNB")),
    ("SGD", sgd_classifier("SGD", loss=sgd_loss, random_state=42))
]

In [71]:
model_list_v2 = [
    ("SVM", SVM_search_space, SVC(random_state=42)),
    ("KNN", KNN_search_space, KNeighborsClassifier(n_jobs=-1)),
    ("Logistic Regression", LR_search_space, LogisticRegression(solver="saga", max_iter=1000, random_state=42, n_jobs=-1)),
    ("Random Forest", RF_search_space, RandomForestClassifier()),
    ("MLP", MLP_search_space, MLPClassifier()),
    ("Gaussian NB", NB_search_space, GaussianNB()),
    ("SGD", SGD_search_space, SGDClassifier())
]

# run pipelines

In [38]:
pheme = get_dataset("pheme")
twitter = get_dataset("twitter")
twitter15 = twitter.iloc[:1491]
twitter16 = twitter.iloc[1491:]
weibo = get_dataset("weibo")
weibo = weibo.drop([1933, 3564])

In [79]:
tests = [["PHEME", "pheme", pheme], ["twitter15", "twitter", twitter15], ["twitter16", "twitter", twitter16]]

In [39]:
tests2 = [["PHEME", "pheme", pheme], ["twitterFULL", "twitter", twitter], ["WEIBO", "weibo", weibo]]

In [14]:
coverage1, models1, results1 = run_tests(tests2, 0.2, 200, model_list)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 1/1 [02:01<00:00, 121.73s/trial, best loss=?]
Error training Random Forest 

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


100%|██████████| 1/1 [02:01<00:00, 121.69s/trial, best loss=?]
Error training Random Forest in category Law & Government, skipping
100%|██████████| 15/15 [00:01<00:00,  1.71s/trial, best loss: 0.16428571428571426]
Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.85s/trial, best loss: 0.23076923076923073]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


...

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.61s/trial, best loss: 0.1711711711711712]
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.63s/trial, best loss: 0.2142857142857143]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.60s/trial, best loss: 0.2142857142857143]
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos & Vehicles due to low numbers


100%|██████████| 3/3 [2:13:59<00:00, 2679.76s/it]


In [15]:
print(coverage1, models1, results1)

[('PHEME', 95.11436128831492), ('twitterFULL', 81.94805194805195), ('WEIBO', 62.00686106346484)] [('PHEME', {'Sensitive Subjects': [0.8252032520325203, 'SVC', SVC(C=1.0258916413641384, coef0=0.9206867755576156,
    decision_function_shape='ovo', degree=4, kernel='poly', random_state=42,
    shrinking=False, tol=0.0004623594172784479)], 'News': [0.7995049504950495, 'Random Forest', RandomForestClassifier(max_features=0.3991314144574376, min_samples_leaf=3,
                       n_estimators=256, n_jobs=1, random_state=42,
                       verbose=False)], 'Arts & Entertainment': [0.9112426035502958, 'SVC', SVC(C=1.099249272359582, coef0=0.3560236746101211, kernel='poly',
    random_state=42, shrinking=False, tol=0.0012720466533368403)], 'People & Society': [0.8853503184713376, 'KNN', KNeighborsClassifier(algorithm='ball_tree', leaf_size=34, metric='cityblock',
                     n_jobs=1, n_neighbors=12, p=3.475006381511173,
                     weights='distance')], 'Law & Gov

In [20]:
coverage2, models2, results2 = run_tests(tests2, 0.5, 0, model_list)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:01<00:00,  1.75s/trial, best loss: 0.1777777777777778]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.74s/trial, best loss: 0.11111111111111116]


...

job exception: Expected n_neighbors <= n_samples,  but n_samples = 4, n_neighbors = 11

 67%|██████▋   | 2/3 [2:46:21<46:16, 2776.84s/it]

  0%|          | 0/1 [00:01<?, ?trial/s, best loss=?]
Error training KNN in category Home & Garden, skipping
100%|██████████| 15/15 [00:01<00:00,  1.66s/trial, best loss: 0.5]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (185) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (185) reached and the optimization hasn't converged yet.
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.64s/trial, best loss: 0.09090909090909094]


100%|██████████| 3/3 [2:51:53<00:00, 3437.96s/it]


In [21]:
print(coverage2, results2)

[('PHEME', 60.94600902442819), ('twitterFULL', 61.298701298701296), ('WEIBO', 43.22469982847341)] [('PHEME', [('PHEME', (72.84, 64.67)), ('twitterFULL', (52.47, 43.7)), ('WEIBO', (52.08, 40.2))]), ('twitterFULL', [('twitterFULL', (62.77, 62.76)), ('PHEME', (55.6, 54.42)), ('WEIBO', (48.86, 48.67))]), ('WEIBO', [('WEIBO', (73.63, 73.43)), ('PHEME', (58.01, 49.45)), ('twitterFULL', (48.92, 46.91))])]


In [22]:
coverage3, models3, results3 = run_tests(tests2, 0.3, 50, model_list)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 1/1 [02:01<00:00, 121.49s/trial, best loss=?]
Error training Random Forest in category News, skipping
100%|██████████| 

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.65s/trial, best loss: 0.08974358974358976]
Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.61s/trial, best loss: 0.33333333333333337]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.66s/trial, best loss: 0.05882352941176472]
Skipped category: Computers & Electronics due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.71s/trial, best loss: 0.2727272727272727]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.76s/trial, best loss: 0.11111111111111116]
Skipped category: Beauty & Fitness due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.71s/trial, best loss: 0.0714285714285714]
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.72s/trial, best loss: 0.16666666666666663]


100%|██████████| 3/3 [2:45:58<00:00, 3319.42s/it]


In [23]:
print(coverage3, results3)

[('PHEME', 87.66142834915202), ('twitterFULL', 76.1038961038961), ('WEIBO', 65.03001715265866)] [('PHEME', [('PHEME', (81.32, 79.15)), ('twitterFULL', (54.64, 49.04)), ('WEIBO', (50.84, 37.11))]), ('twitterFULL', [('twitterFULL', (69.7, 69.53)), ('PHEME', (52.53, 50.12)), ('WEIBO', (52.4, 49.02))]), ('WEIBO', [('WEIBO', (75.88, 75.87)), ('PHEME', (47.74, 47.12)), ('twitterFULL', (53.16, 53.14))])]


In [24]:
coverage4, models4, results4 = run_tests(tests2, 0, 100, model_list)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 1/1 [02:01<00:00, 121.64s/trial, best loss=?]
Error training Random Forest in category Sensitive Subjects, skipping
100%|██████████| 15/15 [00:01<00:00,  1.99s/trial, best loss: 0.2572614107883817]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.97s/trial, best loss: 0.15600000000000003]
Skipped category: Books & Literature due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.78s/trial, best loss: 0.2142857142857143]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


...

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.86s/trial, best loss: 0.1428571428571429]
Skipped category: Beauty & Fitness due to low numbers
100%|██████████| 15/15 [00:01<00:00,  1.98s/trial, best loss: 0.21052631578947367]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 15/15 [00:01<00:00,  1.87s/trial, best loss: 0.2142857142857143]
Skipped category: Home & Garden due to low numbers
100%|██████████| 5/5 [00:02<00:00,  2.58s/trial, best loss: 0.16000000000000003]


In [ ]:
print(coverage4, results4)

In [ ]:
# Multi baseline
from sklearn.ensemble import VotingClassifier

def evaluate_baseline(model, X_test, y_test):
    pred_y = model.predict(X_test)
    acc_mod = accuracy_score(y_test, pred_y)
    f1_mod = f1_score(y_test, pred_y, average="macro")
    return float("{0:.2f}".format(acc_mod*100)), float("{0:.2f}".format(f1_mod*100))


def run_baseline_tests(tests, model_list):
    test_results = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        test_results.append((train[0], get_baseline_results(train[2], tests, model_list)))
    return test_results

def get_baseline_results(train, tests, model_list): 
    X = train.drop("target", axis=1)
    y = train["target"]
    X_train, X_val, y_train, y_val = train_test_split(train.drop("target", axis=1), train["target"], train_size=0.8, stratify=train["target"]) 
    X_train_text = np.array([text for text in X_train['e_text']])
    X_val_text = np.array([text for text in X_val['e_text']])
    models = []
    for model_name, model in model_list:
        try:
            baseline_model = optimize_model(model, X_train_text, y_train, 5)
        except:
            print(f"Error training {model_name}")
            continue
        sk_model = baseline_model.best_model()["learner"].fit(X_train_text, y_train)
        models.append((model_name, sk_model))

    vc_hard = VotingClassifier(estimators=models, voting="hard")
    vc_hard = vc_hard.fit(X_train_text, y_train)

    results = []
    results.append((evaluate_baseline(vc_hard, X_val_text, y_val)))
    for test_name, test_cat, test_set in tests:
        test_data = test_set.drop("target", axis=1)
        test_data_text = np.array([text for text in test_data['e_text']])
        test_target = test_set["target"]
        results.append((test_name, evaluate_baseline(vc_hard, test_data_text, test_target)))
    return results

multi_baseline_results = run_baseline_tests(tests2, model_list)

# Alt method

In [72]:
def train_and_evaluate_v2(train_set, test_set, confidence, size_limit, model_list):
    """
    Train and evaluate dataset on test set. Returns results and models
    train_set: training set. In the form ["dataset_name", file_reference_name, dataset]
    test_set: testing set(s). In the form [("dataset_name1", file_reference_name1, dataset1), ("dataset_name2", file_reference_name2, dataset2), etc]
    confidence: confidence threshold to consider a training point
    size_limit: size threshold to consider training a category
    model_list: list of models to train, in the form [("model_name1", model1), ("model_name2", model2), etc]
    """
    #prep dataset
    dataset_name = train_set[0]
    ref_name = train_set[1]
    dataset = train_set[2]
    X = dataset.drop("target", axis=1)
    y = dataset["target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
    training_set = pd.concat([X_train, y_train], axis=1)

    #get dataset coverage
    info, coverage = get_category_info(ref_name, dataset, confidence, size_limit)
    dataset_coverage = coverage[True]

    #train models
    trained_models = train_models_v2(ref_name, training_set, confidence, size_limit, model_list)

    #make predictions on test set
    test_cat_file = f"{ref_name}_categories.json"
    predictions = predict_points(trained_models, test_cat_file, X_test)
    results = check_score(y_test, predictions)

    final_results = []
    final_results.append((dataset_name, results))

    #make predictions on out of domain test sets
    for set_name, ref_name, set_data in test_set:
        test_cat_file = f"{ref_name}_categories.json"
        X = set_data.drop("target", axis=1)
        y = set_data["target"]
        predictions = predict_points(trained_models, test_cat_file, set_data)
        results = check_score(y, predictions)
        final_results.append((set_name, results))
    
    return dataset_coverage, trained_models, final_results

def run_tests_v2(tests, confidence, size_limit, model_list):
    dataset_coverage = []
    test_results = []
    trained_models = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        coverage, models, results = train_and_evaluate_v2(train, t, confidence, size_limit, model_list)
        dataset_coverage.append((train[0], coverage))
        test_results.append((train[0], results))
        trained_models.append((train[0], models))
    return dataset_coverage, trained_models, test_results

In [76]:
results1 = run_tests_v2(tests2, 0.2, 200, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
 92%|█████████▎| 185/200 [02:00<00:09,  1.54trial/s, best loss: -0.7682926829268293]
Trained

 33%|███▎      | 1/3 [45:21<1:30:42, 2721.46s/it]

Skipped category: Food & Drink due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Health due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden

 67%|██████▋   | 2/3 [1:15:13<36:14, 2174.46s/it]

Skipped category: Reference due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Online Communities due to low numbers
Skipped category: Science due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos

100%|██████████| 3/3 [1:45:35<00:00, 2111.95s/it]


In [77]:
results2  = run_tests_v2(tests2, 0.2, 200, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 200/200 [01:17<00:00,  2.57trial/s, best loss: -0.7655826558265583]
Trained

 33%|███▎      | 1/3 [43:53<1:27:46, 2633.34s/it]

Skipped category: Food & Drink due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Health due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden

 67%|██████▋   | 2/3 [1:10:41<33:50, 2030.37s/it]

Skipped category: Reference due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Online Communities due to low numbers
Skipped category: Science due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos

100%|██████████| 3/3 [1:41:35<00:00, 2031.72s/it]


In [78]:
results3 = run_tests_v2(tests2, 0.2, 200, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 200/200 [00:52<00:00,  3.81trial/s, best loss: -0.7669376693766937]
Trained

 33%|███▎      | 1/3 [45:51<1:31:42, 2751.37s/it]

Skipped category: Food & Drink due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Health due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden

 67%|██████▋   | 2/3 [1:13:32<35:10, 2110.15s/it]

Skipped category: Reference due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Online Communities due to low numbers
Skipped category: Science due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos

100%|██████████| 3/3 [1:43:23<00:00, 2067.96s/it]


In [79]:
results4 = run_tests_v2(tests2, 0.2, 200, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 200/200 [00:57<00:00,  3.48trial/s, best loss: -0.7669376693766937]
Trained

 33%|███▎      | 1/3 [43:10<1:26:21, 2590.78s/it]

Skipped category: Food & Drink due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Health due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden

 67%|██████▋   | 2/3 [1:10:06<33:37, 2017.10s/it]

Skipped category: Reference due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Online Communities due to low numbers
Skipped category: Science due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos

In [41]:
models = train_models_v2("pheme", pheme, 0.2, 200, model_list_v2)

100%|██████████| 2/2 [00:00<00:00,  3.77trial/s, best loss: -0.6073752711496746]
Trained models on Sensitive Subjects, added SVM to list
100%|██████████| 2/2 [00:00<00:00,  4.32trial/s, best loss: -0.7455968688845401]
Trained models on News, added KNN to list
100%|██████████| 2/2 [00:00<00:00,  7.92trial/s, best loss: -0.8436018957345972]
Trained models on Arts & Entertainment, added KNN to list
100%|██████████| 2/2 [00:00<00:00, 38.95trial/s, best loss: -0.8691099476439791]
Trained models on People & Society, added KNN to list
100%|██████████| 2/2 [00:00<00:00, 51.06trial/s, best loss: -0.5584905660377358]
Trained models on Law & Government, added SVM to list
100%|██████████| 2/2 [00:00<00:00, 19.72trial/s, best loss: -0.852017937219731]
Trained models on Online Communities, added KNN to list
Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low nu

In [73]:
test = twitter.drop("target", axis=1)
pred = predict_points(models, "twitter_categories.json", test)

In [75]:
r = check_score(pred, twitter['target'])
r

(49.48, 49.36)

In [42]:
model_list_v3 = [
    ("SVM", SVM_search_space, SVC(random_state=42)),
    ("MLP", MLP_search_space, MLPClassifier()),
    ("SGD", SGD_search_space, SGDClassifier())
]

In [49]:
results1 = run_tests_v2(tests2, 0.2, 200, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 1/1 [00:20<00:00, 20.07s/trial, best loss=?]
Error training KNN in category

 33%|███▎      | 1/3 [13:53<27:46, 833.44s/it]

Skipped category: Food & Drink due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Health due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden

 67%|██████▋   | 2/3 [20:53<09:50, 590.48s/it]

Skipped category: Reference due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Games due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Online Communities due to low numbers
Skipped category: Science due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos

100%|██████████| 3/3 [31:36<00:00, 632.31s/it]


In [50]:
results1[2]

[('PHEME',
  [('PHEME', (79.77, 77.97)),
   ('twitterFULL', (55.72, 50.86)),
   ('WEIBO', (50.47, 36.13))]),
 ('twitterFULL',
  [('twitterFULL', (73.59, 73.52)),
   ('PHEME', (57.39, 55.94)),
   ('WEIBO', (49.14, 45.1))]),
 ('WEIBO',
  [('WEIBO', (73.74, 73.22)),
   ('PHEME', (57.2, 46.24)),
   ('twitterFULL', (50.04, 44.09))])]

In [37]:
results2 = run_tests_v2(tests2, 0.5, 5, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 20/20 [00:07<00:00,  2.68trial/s, best loss: -0.7891050583657587]
Trained models on Sensitive Subjects, added SVM to list
100%|██████████| 20/20 [00:15<00:00,  1.30trial/s, best loss: -0.7828793774319066]
Trained models on News, added SVM to list
100%|██████████| 20/20 [00:09<00:00,  2.12trial/s, best loss: -0.7867704280155642]
Trained models on Arts & Entertainment, added

 33%|███▎      | 1/3 [27:51<55:42, 1671.33s/it]

Skipped category: Adult due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Real Estate due to low numbers
100%|██████████| 20/20 [00:15<00:00,  1.29trial/s, best loss: -0.7844357976653696]
Trained models on People & Society, added SVM to list
100%|██████████| 20/20 [00:29<00:00,  1.46s/trial, best loss: -0.7867704280155642]
Trained models on Arts & Entertainment, added MLP to list
100%|██████████| 20/20 [00:09<00:00,  2.10trial/s, best loss: -0.7844357976653696]
Trained models on Law & Government, added SVM to list
100%|██████████| 20/20 [00:19<00:00,  1.04trial/s, best loss: -0.7836575875486381]
Trained models on News, added MLP to list
100%|██████████| 20/20 [00:15<00:00,  1.26trial/s, best loss: -0.7875486381322957]
Trained models on Food & Drink, added MLP to list
100%|██████████| 20/20 [00:38<00:00,  1.95s/trial, best loss: -0.7867704280155642]
Tr

 67%|██████▋   | 2/3 [1:09:27<35:56, 2156.29s/it]

100%|██████████| 20/20 [00:17<00:00,  1.16trial/s, best loss: -0.7852140077821012]
Trained models on Arts & Entertainment, added SVM to list
100%|██████████| 20/20 [00:06<00:00,  3.15trial/s, best loss: -0.7852140077821012]
Trained models on People & Society, added SVM to list
100%|██████████| 20/20 [00:09<00:00,  2.18trial/s, best loss: -0.7813229571984436]
Trained models on Reference, added MLP to list
100%|██████████| 20/20 [00:18<00:00,  1.06trial/s, best loss: -0.7883268482490272]
Trained models on Food & Drink, added SVM to list
100%|██████████| 20/20 [00:09<00:00,  2.17trial/s, best loss: -0.7813229571984436]
Trained models on Sports, added SVM to list
100%|██████████| 20/20 [00:18<00:00,  1.09trial/s, best loss: -0.7867704280155642]
Trained models on Games, added SVM to list
100%|██████████| 20/20 [00:18<00:00,  1.09trial/s, best loss: -0.7906614785992218]
Trained models on Travel & Transportation, added SVM to list
100%|██████████| 20/20 [00:10<00:00,  1.91trial/s, best loss: 

100%|██████████| 3/3 [2:21:41<00:00, 2833.76s/it]


In [35]:
results3 = run_tests_v2(tests2, 0.3, 50, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 20/20 [00:12<00:00,  1.61trial/s, best loss: -0.7852140077821012]
Trained models on Sensitive Subjects, added MLP to li

 33%|███▎      | 1/3 [24:56<49:52, 1496.23s/it]

Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Real Estate due to low numbers
100%|██████████| 20/20 [00:09<00:00,  2.15trial/s, best loss: -0.7852

 67%|██████▋   | 2/3 [51:15<25:45, 1545.26s/it]

Skipped category: Games due to low numbers
Skipped category: Online Communities due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
100%|██████████| 20/20 [00:24<00:00,  1.22s/trial, best loss: -0.7836575875486381]
Trained models on Arts & Entertainment, added MLP to list
100%|██████████| 20/20 [00:20<00:00,  1.01s/trial, best loss: -0.7898832684824902]
Trained models on People & Society, added SVM to list
100%|██████████| 20/20 [00:18<00:00,  1.10trial/s, best loss: -0.7828793774319066]
Trained models on Reference, added SVM to list
100%|██████████| 20/20 [00:06<00:00,  3.16trial/s, best loss: -0.7867704280155642]
Trained models on Food & Drink, added SVM to list
100%|███████

100%|██████████| 3/3 [1:44:21<00:00, 2087.32s/it]


In [36]:
results4 = run_tests_v2(tests2, 0, 100, model_list_v2)

  0%|          | 0/3 [00:00<?, ?it/s]

Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
100%|██████████| 20/20 [00:18<00:00,  1.11trial/s, best loss: -0.7844357976653696]
Trained models on Sensitive Subjects, added SVM to list
100%|██████████| 20/20 [00:11<00:00,  1.76trial/s, best loss: -0.7891050583657587]
Trained models on Ne

 33%|███▎      | 1/3 [30:38<1:01:16, 1838.26s/it]

Skipped category: Internet & Telecom due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Science due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Travel & Transportation due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Real Estate due to low numbers
100%|██████████| 20/20 [00:13<00:00,  1.52trial/s, best loss: -0.7828793774319066]
Trained models on People & Society, added MLP

 67%|██████▋   | 2/3 [1:00:31<30:11, 1811.81s/it]

Skipped category: Games due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Home & Garden due to low numbers
100%|██████████| 20/20 [00:15<00:00,  1.27trial/s, best loss: -0.7859922178988327]
Trained models on Arts & Entertainment, added MLP to list
100%|██████████| 20/20 [00:14<00:00,  1.36trial/s, best loss: -0.7836575875486381]
Trained models on People & Society, added SVM to list
100%|██████████| 20/20 [00:14<00:00,  1.42trial/s, best loss: -0.7852140077821012]
Trained models on Reference, added MLP to list
100%|██████████| 20/20 [00:04<00:00,  4.30trial/s, best loss: -0.7813229571984436]
Trained models on Food & Drink, added SVM to list
100%|██████████| 20/20 [00:09<00:00,  2.17trial/s, best loss: -0.7844357976653696]
Trained models on Sports, added KNN to list
Skipped category: Games due t

100%|██████████| 3/3 [2:01:38<00:00, 2432.70s/it]


In [44]:
print(results1[1])

[('PHEME', {'Sensitive Subjects': [0.8404669260700389, 'MLP', MLPClassifier(alpha=0.0013841673676165446, beta_1=0.8903402233581311,
              beta_2=0.9569337314275386, epsilon=9.156355869927632e-06,
              learning_rate_init=0.044564129651472446,
              momentum=0.9332512027206787, power_t=0.5794904340559764,
              solver='sgd', tol=0.004248433341202511)], 'News': [0.8428015564202335, 'SVM', SVC(C=6.712645138459076, coef0=0.16558431993335376, degree=1, random_state=42,
    shrinking=False, tol=0.00138052701904096)], 'Arts & Entertainment': [0.8521400778210116, 'SVM', SVC(C=6.712645138459076, coef0=0.16558431993335376, degree=1, random_state=42,
    shrinking=False, tol=0.00138052701904096)], 'People & Society': [0.8350194552529183, 'MLP', MLPClassifier(alpha=0.0013841673676165446, beta_1=0.8903402233581311,
              beta_2=0.9569337314275386, epsilon=9.156355869927632e-06,
              learning_rate_init=0.044564129651472446,
              momentum=0.93